# Deskripsi Proyek

membangun model yang dapat memprediksi credit risk menggunakan dataset yang disediakan oleh company yang terdiri dari data pinjaman yang diterima dan yang ditolak.

Datasets: loan_data_2007_2014

## Library

In [42]:
import pandas as pd, numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import OneHotEncoder

## Pre-Processing

In [43]:
path = '/home/jovyan/work/IDx_Partner/loan_data_2007_2014.csv'

In [44]:
df = pd.read_csv(path,index_col = 0, low_memory=False)

In [45]:
df.sample()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
249409,35024288,37707601,25000,25000,25000.0,36 months,11.44,823.69,B,B4,...,NaN,NaN,NaN,NaN,NaN,NaN,18100.0,NaN,NaN,NaN


In [46]:
df.shape

(466285, 74)

In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 466285 entries, 0 to 466284
Data columns (total 74 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   id                           466285 non-null  int64  
 1   member_id                    466285 non-null  int64  
 2   loan_amnt                    466285 non-null  int64  
 3   funded_amnt                  466285 non-null  int64  
 4   funded_amnt_inv              466285 non-null  float64
 5   term                         466285 non-null  object 
 6   int_rate                     466285 non-null  float64
 7   installment                  466285 non-null  float64
 8   grade                        466285 non-null  object 
 9   sub_grade                    466285 non-null  object 
 10  emp_title                    438697 non-null  object 
 11  emp_length                   445277 non-null  object 
 12  home_ownership               466285 non-null  object 
 13 

In [48]:
df.describe()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,...,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
count,4.662850e+05,4.662850e+05,466285.000000,466285.000000,466285.000000,466285.000000,466285.000000,4.662810e+05,466285.000000,466256.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,3.960090e+05,0.0,0.0,0.0
mean,1.307973e+07,1.459766e+07,14317.277577,14291.801044,14222.329888,13.829236,432.061201,7.327738e+04,17.218758,0.284678,...,NaN,NaN,NaN,NaN,NaN,NaN,3.037909e+04,NaN,NaN,NaN
std,1.089371e+07,1.168237e+07,8286.509164,8274.371300,8297.637788,4.357587,243.485550,5.496357e+04,7.851121,0.797365,...,NaN,NaN,NaN,NaN,NaN,NaN,3.724713e+04,NaN,NaN,NaN
min,5.473400e+04,7.047300e+04,500.000000,500.000000,0.000000,5.420000,15.670000,1.896000e+03,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,NaN
25%,3.639987e+06,4.379705e+06,8000.000000,8000.000000,8000.000000,10.990000,256.690000,4.500000e+04,11.360000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,1.350000e+04,NaN,NaN,NaN
50%,1.010790e+07,1.194108e+07,12000.000000,12000.000000,12000.000000,13.660000,379.890000,6.300000e+04,16.870000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,2.280000e+04,NaN,NaN,NaN
75%,2.073121e+07,2.300154e+07,20000.000000,20000.000000,19950.000000,16.490000,566.580000,8.896000e+04,22.780000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,3.790000e+04,NaN,NaN,NaN
max,3.809811e+07,4.086083e+07,35000.000000,35000.000000,35000.000000,26.060000,1409.990000,7.500000e+06,39.990000,29.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,9.999999e+06,NaN,NaN,NaN


### Informasi setiap Fitur

In [49]:
#Melakukan pengecekan value setiap fitur
#yang dicantumkan di notebook ini sebagai contoh saya ambil dari purpose dan grade
df['purpose'].value_counts()

debt_consolidation    274195
credit_card           104157
home_improvement       26537
other                  23690
major_purchase          9828
small_business          7013
car                     5397
medical                 4602
moving                  2994
vacation                2487
wedding                 2343
house                   2269
educational              422
renewable_energy         351
Name: purpose, dtype: int64

In [50]:
df['grade'].value_counts()

B    136929
C    125293
D     76888
A     74867
E     35757
F     13229
G      3322
Name: grade, dtype: int64

Selain dari 2 sample di atas, dilakukan juga pengecekan terhadap nilai seluruh fitur yang ada dan didapatkan kesimpulan beberapa kolom yang tidak memiliki value akan di hapus dan hanya fitur yang diperlukan dalam hal modeling yang akan di pertahan kan.

kolom **id** disini akan dijadikan index.

### Menghapus fitur yang tidak diperlukan

Fitur yang akan di gunakan adalah sebagai berikut:

index = id
1. loan_amnt = jumlah peminjaman
2. term = lama nya peminjaman (36 bulan dan 60 bulan)
3. int_rate = suku bunga pinjaman
4. installment = pembayaran bulanan
5. grade = LC menilai pinjaman (A - G)
6. subgrade = subgrade dari pinjaman (_1 - _5)
7. empt_length = lama bekerja (0-10+)
8. home_ownership =status kepemilikan rumah
9. annual_inc = pendapatan tahunan
10. loan_status = status pinjaman saat ini
11. purpose = kepentingan pinjaman
12. addr_state = negara dari peminjam
13. dti = Rasio dari hutang bulanan terhadap total utang dibagi dengan pendapatan bulanan
14. delinq_2yrs = Jumlah insiden tunggakan selama lebih dari 30 hari dalam catatan kredit peminjam selama 2 tahun terakhir.
15. inq_last_6mths
16. open_acc = Jumlah garis kredit yang masih aktif dalam catatan kredit peminjam.
17. pub_rec = Jumlah catatan publik yang merugikan
18. revol_bal = Total saldo kredit yang dapat diputar
19. initial_list_status = Status penulisan awal dari pinjaman. Nilai yang mungkin adalah - "Whole" (Utuh) atau "Fractional" (Fraksional).
20. collection_recovery_fee = biaya penagihan setelah hutang di-charge off
21. acc_now_delinq = Jumlah akun di mana peminjam saat ini mengalami keterlambatan pembayaran.

In [51]:
columns = ['id','loan_amnt','term', 'int_rate', 'installment', 'grade', 'sub_grade',
           'emp_length', 'home_ownership', 'annual_inc','loan_status','purpose','addr_state','dti',
           'delinq_2yrs','inq_last_6mths','open_acc', 
           'pub_rec', 'revol_bal','initial_list_status','collection_recovery_fee',
           'acc_now_delinq']

In [52]:
df_model = df[columns].set_index('id')
df_model.sample()

,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,loan_status,...,addr_state,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,initial_list_status,collection_recovery_fee,acc_now_delinq
id,,,,,,,,,,,,,,,,,,,,,
34393839,23975,60 months,11.44,526.56,B,B4,NaN,RENT,50000.0,Current,...,CA,28.3,0.0,0.0,10.0,0.0,22787,w,0.0,0.0


### Memperbaiki nilai

#### term

In [53]:
#cek value dari term
df_model['term'].unique()

array([' 36 months', ' 60 months'], dtype=object)

In [54]:
# merubah nilai dari str ke int
df_model['term'] = df_model['term'].str.replace('months', '').astype(int)

In [55]:
df_model['term'].unique()

array([36, 60])

##### emp_length

In [56]:
#cek value dari emp_length
df_model['emp_length'].unique()

array(['10+ years', '< 1 year', '1 year', '3 years', '8 years', '9 years',
       '4 years', '5 years', '6 years', '2 years', '7 years', nan],
      dtype=object)

In [57]:
#mengubah nilai nan dan <1 year menjadi 0 sesuai ketentuan LCDataDictionary RejectStats
df_model.loc[(df_model['emp_length'].isna()) | (df_model['emp_length'] == '< 1 year'),'emp_length']= 0

In [58]:
#mengubah tipe data emp__length dari string menjadi integer
df_model['emp_length'] = df_model['emp_length'].apply(lambda x: re.findall('([0-9]+)',str(x))[0]).astype(int)

In [59]:
df_model['emp_length'].value_counts()

10    150049
0      57273
2      41373
3      36596
5      30774
1      29622
4      28023
7      26180
6      26112
8      22395
9      17888
Name: emp_length, dtype: int64

emp_length adalah Panjang pekerjaan dalam beberapa tahun. Nilai yang mungkin adalah antara 0 sampai 10 di mana 0 berarti kurang dari satu tahun dan 10 berarti sepuluh tahun atau lebih.

##### anual_inc

In [60]:
#Cek jumlah nilai yang hilang
df_model['annual_inc'].isna().sum()

4

In [74]:
#menghapus baris yang memiliki nilai hilang di annual_inc
df_model = df_model.dropna(subset = ['annual_inc'])

Dikarenakan jumlah missing value dari annual_inc relatif kecil dan penghapusan ini tidak mengurangi representativitas dataset secara signifikan.

##### home_ownership

In [62]:
#unique value dari ownership
df_model['home_ownership'].unique()

array(['RENT', 'OWN', 'MORTGAGE', 'OTHER', 'NONE', 'ANY'], dtype=object)

In [63]:
df_model['home_ownership'].value_counts()

MORTGAGE    235875
RENT        188473
OWN          41704
OTHER          182
NONE            50
ANY              1
Name: home_ownership, dtype: int64

In [64]:
#merubah nilai None dan ANY menjadi OTHER
df_model.loc[(df_model['home_ownership'] == 'NONE') | 
       (df_model['home_ownership'] == 'ANY'),'home_ownership']= 'OTHER'

Dikarenakan nilai NONE dan ANY hanya berjumlah sedikit dan cenderung memiliki makna yang hampir mirip dengan OTHER, oleh karena itu nilai tersebut digabungkan menjadi OTHER.

##### Loan Status

Rincian skor kredit berdasarkan BI Checking:
1. Skor 1: Kredit Lancar, artinya debitur selalu memenuhi kewajibannya untuk membayar cicilan setiap bulan beserta bunganya hingga lunas tanpa pernah menunggak.
2. Skor 2: Kredit DPK atau Kredit dalam Perhatian Khusus, artinya debitur tercatat menunggak cicilan kredit 1-90 hari
3. Skor 3: Kredit Tidak Lancar, artinya debitur tercatat menunggak cicilan kredit 91-120 hari
4. Skor 4: Kredit Diragukan, artinya debitur tercatat menunggak cicilan kredit 121-180 hari
5. Skor 5: Kredit Macet, artinya debitur tercatat menunggak cicilan kredit lebih 180 hari.

In [65]:
df_model['loan_status'].value_counts()

Current                                                224226
Fully Paid                                             184739
Charged Off                                             42475
Late (31-120 days)                                       6900
In Grace Period                                          3146
Does not meet the credit policy. Status:Fully Paid       1988
Late (16-30 days)                                        1218
Default                                                   832
Does not meet the credit policy. Status:Charged Off       761
Name: loan_status, dtype: int64

Dari keterangan loan status di dapat 
1. skor 1 = Current dan Fully Paid
2. skor 2 = In Grace Period , Late (16-30 days)
3. skor 3 = Late (31-120 days),Does not meet the credit policy. Status:Fully Paid
4. skor 4 = Charged Off, Does not meet the credit policy. Status:Charged Off 
5. skor 5 = Default

In [66]:
#Merubah Loan status berdasarkan ketentuan di atas dan akan menjadi target dari model kita
mapping = {
    'Current' : 1, 'Fully Paid' : 1,
    'In Grace Period': 2, 'Late (16-30 days)': 2,
    'Late (31-120 days)': 3, 'Does not meet the credit policy. Status:Fully Paid' : 3,
    'Charged Off': 4, 'Does not meet the credit policy. Status:Charged Off' : 4,
    'Default' : 5
}
df_model['loan_status'] = df_model['loan_status'].replace(mapping)

In [67]:
df_model['loan_status'].value_counts()

1    408965
4     43236
3      8888
2      4364
5       832
Name: loan_status, dtype: int64

##### deling_2yrs

In [68]:
df_model[df_model['delinq_2yrs'].isna()]

,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,loan_status,...,addr_state,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,initial_list_status,collection_recovery_fee,acc_now_delinq
id,,,,,,,,,,,,,,,,,,,,,
79967,5000,36,7.43,155.38,A,A2,0,OTHER,NaN,3,...,NY,1.00,NaN,NaN,NaN,NaN,0,f,0.0000,NaN
79924,7000,36,7.75,218.55,A,A3,0,OTHER,NaN,3,...,NY,1.00,NaN,NaN,NaN,NaN,0,f,0.0000,NaN
71823,4350,36,8.07,136.45,A,A4,0,RENT,120000.0,3,...,MA,4.00,NaN,NaN,NaN,NaN,0,f,0.0000,NaN
72197,3900,36,9.33,124.62,B,B3,0,RENT,98000.0,3,...,NY,10.00,NaN,NaN,NaN,NaN,0,f,0.0000,NaN
79893,6700,36,7.75,209.18,A,A3,0,OTHER,NaN,3,...,NY,1.00,NaN,NaN,NaN,NaN,0,f,0.0000,NaN
73003,3200,36,9.96,103.20,B,B5,0,MORTGAGE,150000.0,3,...,MA,10.00,NaN,NaN,NaN,NaN,0,f,0.0000,NaN
72819,7200,36,10.28,164.42,C,C1,0,RENT,95000.0,3,...,NY,10.00,NaN,NaN,NaN,NaN,0,f,0.0000,NaN
91023,1900,36,9.64,61.00,B,B4,1,MORTGAGE,100000.0,3,...,FL,10.00,NaN,NaN,NaN,NaN,0,f,0.0000,NaN
85961,1200,36,9.01,38.17,B,B2,0,RENT,36000.0,3,...,TX,3.27,NaN,NaN,NaN,NaN,0,f,0.0000,NaN


In [73]:
df_model = df_model.dropna(subset = ['delinq_2yrs'])

Dikarenakan jumlah missing value dari delinq_2yrs relatif kecil dan penghapusan ini tidak mengurangi representativitas dataset secara signifikan.

In [75]:
df_model.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 466256 entries, 1077501 to 9199665
Data columns (total 21 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   loan_amnt                466256 non-null  int64  
 1   term                     466256 non-null  int64  
 2   int_rate                 466256 non-null  float64
 3   installment              466256 non-null  float64
 4   grade                    466256 non-null  object 
 5   sub_grade                466256 non-null  object 
 6   emp_length               466256 non-null  int64  
 7   home_ownership           466256 non-null  object 
 8   annual_inc               466256 non-null  float64
 9   loan_status              466256 non-null  int64  
 10  purpose                  466256 non-null  object 
 11  addr_state               466256 non-null  object 
 12  dti                      466256 non-null  float64
 13  delinq_2yrs              466256 non-null  float64
 1

## Eksplorasi Data Analisis

### Dataset Supervised

### Modeling

#### Data Modeling

In [ ]:
df_supervised.sample()

In [ ]:
df_unsupervised.sample()

In [ ]:
enc = OneHotEncoder(handle_unknown='ignore')

In [ ]:
onehot_encoded = enc.fit_transform(df_unsupervised[['initial_list_status']]).toarray()
encoded_df_unsupervised = pd.DataFrame(onehot_encoded, columns=enc.get_feature_names_out(['initial_list_status']))

In [ ]:
onehot_encoded

In [ ]:
df_unsupervised.sample()

In [ ]:
K = range(1,10)
distortion = []
for k in K:
    # < ketik kode program Anda di sini >
    model = KMeans(n_clusters=k, random_state=12345)
    model.fit(df_unsupervised)
    distortion.append(model.inertia_)
plt.figure(figsize=(12, 3))
plt.plot(K, distortion, 'bx-')
plt.xlabel('Jumlah klaster')
plt.ylabel('Nilai fungsi objektif')
plt.title('Optimal klaster')
plt.show()